#基于注意力的神经机器翻译

此笔记本训练一个将汉语翻译为英语的序列到序列（sequence to sequence，简写为 seq2seq）模型。此例子难度较高，需要对序列到序列模型的知识有一定了解。

训练完此笔记本中的模型后，你将能够输入一个汉语句子，例如 "你今天早晨吃的啥?"，并返回其英语翻译 "are you still at home?"

对于一个简单的例子来说，翻译质量令人满意。但是更有趣的可能是生成的注意力图：它显示在翻译过程中，输入句子的哪些部分受到了模型的注意。

![](https://tensorflow.org/images/spanish-english.png)

请注意：运行这个例子用一个 P100 GPU 需要花大约 10 分钟。

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

###下载和准备数据集
我们将使用 http://www.manythings.org/anki/ 提供的一个语言数据集。这个数据集包含如下格式的语言翻译对：

```
I won! 我赢了。
```


这个数据集中有很多种语言可供选择。我们将使用英语 - 汉语数据集。为方便使用，我们在谷歌云上提供了此数据集的一份副本。但是你也可以自己下载副本。下载完数据集后，我们将采取下列步骤准备数据：

1. 给每个句子添加一个 开始 和一个 结束 标记（token）。
2. 删除特殊字符以清理句子。
3. 创建一个单词索引和一个反向单词索引（即一个从单词映射至 id 的词典和一个从 id 映射至单词的词典）。
4. 将每个句子填充（pad）到最大长度。

In [12]:
!wget http://www.manythings.org/anki/cmn-eng.zip

--2020-02-02 04:01:00--  http://www.manythings.org/anki/cmn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:3037::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 941294 (919K) [application/zip]
Saving to: ‘cmn-eng.zip’

cmn-eng.zip         100%[===================>] 919.23K  3.63MB/s    in 0.2s    

2020-02-02 04:01:01 (3.63 MB/s) - ‘cmn-eng.zip’ saved [941294/941294]



In [16]:
%ls -l

total 924
-rw-r--r-- 1 root root 941294 Jan 11 14:48 cmn-eng.zip
drwxr-xr-x 1 root root   4096 Jan 13 16:38 sample_data/


下载有问题，先将文件手工下载，然后将它移动到缓存中去

In [0]:
!mv cmn-eng.zip ~/.keras/datasets/

In [0]:
# 下载文件
path_to_zip = tf.keras.utils.get_file(
    'cmn-eng.zip',origin='http://www.manythings.org/anki/cmn-eng.zip',
    extract=True
)

path_to_file = os.path.dirname(path_to_zip)+'/cmn-eng/cmn.txt'

In [29]:
%ls ~/.keras/datasets/

_about.txt  cmn-eng.zip  cmn.txt


In [23]:
!head ~/.keras/datasets/cmn.txt

Hi.	嗨。	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #891077 (Martha)
Hi.	你好。	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4857568 (musclegirlxyp)
Run.	你用跑的。	CC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #3748344 (egg0073)
Wait!	等等！	CC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #4970122 (wzhd)
Wait!	等一下！	CC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #5092613 (mirrorvan)
Hello!	你好。	CC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #4857568 (musclegirlxyp)
I try.	让我来。	CC-BY 2.0 (France) Attribution: tatoeba.org #20776 (CK) & #5092185 (mirrorvan)
I won!	我赢了。	CC-BY 2.0 (France) Attribution: tatoeba.org #2005192 (CK) & #5102367 (mirrorvan)
Oh no!	不会吧。	CC-BY 2.0 (France) Attribution: tatoeba.org #1299275 (CK) & #5092475 (mirrorvan)
Cheers!	乾杯!	CC-BY 2.0 (France) Attribution: tatoeba.org #487006 (human600) & #765577 (Martha)


In [0]:
# 将 unicode 文件转换为 ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD',s)
                  if unicodedata.category(c) != 'Mn')
  
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 在单词与跟在其后的标点符号之间插入一个空格
  # 例如： "he is a boy." => "he is a boy ."
  # 参考：https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation

  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # 除了 (a-z, A-Z, ".", "?", "!", ",")，将所有字符替换为空格
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.rstrip().strip()

  # 给句子加上开始和结束标记
  # 以便模型知道何时开始和结束预测
  w = '<start> ' + w + ' <end>'
  return w


In [33]:
en_sentence = u"I won!"
cn_sentence = u"我赢了。"

print(preprocess_sentence(en_sentence))
print(preprocess_sentence(cn_sentence).encode('utf-8'))

<start> i won ! <end>
b'<start>  <end>'


##中文有总是还是返回西班牙语吧

In [30]:
# 下载文件
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

2646016/2638744 [==============================] - 0s 0us/step


In [34]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [0]:
# 1. 去除重音符号
# 2. 清理句子
# 3. 返回这样格式的单词对：[ENGLISH, SPANISH]
def create_dataset(path,num_examples):
  lines = io.open(path,encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]

  return zip(*word_pairs)

In [36]:
en,sp = create_dataset(path_to_file,None)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [0]:
def load_dataset(path,num_examples=None):
  # 创建清理过的输入输出对
  targ_lang,inp_lang = create_dataset(path,num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor,target_tensor,inp_lang_tokenizer,targ_lang_tokenizer

####限制数据集的大小以加快实验速度（可选）
在超过 10 万个句子的完整数据集上训练需要很长时间。为了更快地训练，我们可以将数据集的大小限制为 3 万个句子（当然，翻译质量也会随着数据的减少而降低）：

In [0]:
#尝试实验不同大小的数据集
num_examples = 30000
input_tensor,target_tensor,inp_lang,targ_lang = load_dataset(path_to_file,num_examples)

# 计算目标张量的最大长度 （max_length）
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [41]:
# 采用 80 - 20 的比例切分训练集和验证集
input_tensor_train,input_tensor_val,target_tensor_train,target_tensor_val = train_test_split(input_tensor,target_tensor,test_size=0.2)

# 显示长度
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print("%d ----> %s" % (t,lang.index_word[t]))

In [43]:
print("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print()
print("Target Language; index to word mapping")
convert(targ_lang,target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
143 ----> podria
4112 ----> matarlas
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> i
169 ----> could
302 ----> kill
6 ----> you
3 ----> .
2 ----> <end>


###创建一个 tf.data 数据集

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train,target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)

In [45]:
example_input_batch,example_target_batch = next(iter(dataset))
example_input_batch.shape,example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

###编写编码器 （encoder） 和解码器 （decoder） 模型
实现一个基于注意力的编码器 - 解码器模型。关于这种模型，你可以阅读 TensorFlow 的 神经机器翻译 (序列到序列) 教程 [链接文字](https://github.com/tensorflow/nmt)。本示例采用一组更新的 API。此笔记本实现了上述序列到序列教程中的 [注意力方程式](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism)。下图显示了注意力机制为每个输入单词分配一个权重，然后解码器将这个权重用于预测句子中的下一个单词。下图和公式是 [Luong 的论文](https://arxiv.org/abs/1508.04025v5)中注意力机制的一个例子。

![attention mechanism ](https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg)

输入经过编码器模型，编码器模型为我们提供形状为 (批大小，最大长度，隐藏层大小) 的编码器输出和形状为 (批大小，隐藏层大小) 的编码器隐藏层状态。

下面是所实现的方程式：

![attention equation 0 ](https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg)![attention equation 1](https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg)

本教程的编码器采用 Bahdanau 注意力。在用简化形式编写之前，让我们先决定符号：

* FC = 完全连接（密集）层
* EO = 编码器输出
* H = 隐藏层状态
* X = 解码器输入

以及伪代码：

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)。` Softmax 默认被应用于最后一个轴，但是这里我们想将它应用于 第一个轴, 因为分数 （score） 的形状是 (批大小，最大长度，隐藏层大小)。最大长度 （max_length） 是我们的输入的长度。因为我们想为每个输入分配一个权重，所以 softmax 应该用在这个轴上。
* `context vector = sum(attention weights * EO, axis = 1)`。选择第一个轴的原因同上。
* `embedding output` = 解码器输入 X 通过一个嵌入层。
* `merged vector = concat(embedding output, context vector)`
* 此合并后的向量随后被传送到 GRU
每个步骤中所有向量的形状已在代码的注释中阐明：

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size, embedding_dim,enc_units,batch_sz):
    super(Encoder,self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output,state = self.gru(x,initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz,self.enc_units))

In [47]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# 样本输入
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


#未完成，待续